## Setup

In [1]:
import os, json, math
import re
import numpy as np
import matplotlib.pyplot as plt
import ast
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import efficientnet, EfficientNetV2M, EfficientNetV2L
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import regularizers
import string


seed = 111
np.random.seed(seed)
tf.random.set_seed(seed)

2025-10-05 11:15:04.202855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-10-05 11:15:04.202914: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
with open('../datasets/Flickr30k/dataset_flickr30k_all.json', 'r') as captionsJson:
  captions = json.load(captionsJson)
with open('../datasets/Flickr30k/dataset_flickr30k.json', 'r') as captionsJson:
  info = json.load(captionsJson)

In [3]:
# Path to the images
IMAGES_PATH = "../datasets/Flickr30k/flickr30k-images"

# Desired image dimensions
IMAGE_SIZE = (384, 384)

# Vocabulary size
VOCAB_SIZE = 8249

# Fixed length allowed for any sequence
SEQ_LENGTH = 25

# Other training parameters
BATCH_SIZE = 25
AUTOTUNE = tf.data.AUTOTUNE

## Preparing the dataset

In [4]:

def load_captions_data(captionsfile):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """
    caption_mapping = {}
    text_data = []
    images_to_skip = set()

    for img_name, captions in captionsfile.items():
        img_name = os.path.join(IMAGES_PATH, img_name)
        for caption in captions:
          tokens = caption.strip().split()

          if img_name.endswith("jpg") and img_name not in images_to_skip:
              # We will add a start and an end token to each caption
              # caption = "<start> " + caption.strip() + " <end>"
              text_data.append(caption)

              if img_name in caption_mapping:
                  caption_mapping[img_name].append(caption)
              else:
                  caption_mapping[img_name] = [caption]

    return caption_mapping, text_data


def train_val_split(caption_data, train_n, val_n, test_n, shuffle=True):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())

    # 2. Shuffle if necessary
    if shuffle:
        np.random.shuffle(all_images)

    # 3. Split into training and validation sets
    # train_size = int(len(caption_data) * train_size)
    train_size=train_n
    val_size=val_n
    test_size=test_n

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:train_size+val_size]
    }
    test_data = {
        img_name: caption_data[img_name] for img_name in all_images[-test_size:]
    }

    # 4. Return the splits
    return training_data, validation_data, test_data

def train_val_split_karpathy(caption_data, info_data):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())
    all_images = np.array(all_images)
    train_idx, val_idx, test_idx = [], [], []

    for image in info_data['images']:
      if image['split'] == 'train':
        train_idx.append(np.where(all_images==IMAGES_PATH + '/' + image['filename']))
      if image['split'] == 'val':
        val_idx.append(np.where(all_images==IMAGES_PATH + '/' + image['filename']))
      if image['split'] == 'test':
        test_idx.append(np.where(all_images==IMAGES_PATH + '/' + image['filename']))


    training_data = {
        img_name[0][0]: caption_data[img_name[0][0]] for img_name in all_images[train_idx]
    }
    validation_data = {
        img_name[0][0]: caption_data[img_name[0][0]] for img_name in all_images[val_idx]
    }
    test_data = {
        img_name[0][0]: caption_data[img_name[0][0]] for img_name in all_images[test_idx]
    }

    # 4. Return the splits
    return training_data, validation_data, test_data


# Load the dataset
captions_mapping, text_data = load_captions_data(captions)

# Split the dataset into training and validation sets
# train_data, valid_data, test_data = train_val_split(captions_mapping, 29014, 1000, 1000)
train_data, valid_data, test_data = train_val_split_karpathy(captions_mapping, info)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))
print("Number of test samples: ", len(test_data))

Number of training samples:  29000
Number of validation samples:  1014
Number of test samples:  1000


In [5]:
train_data_text = []
for i in train_data.values(): train_data_text.extend(i)

In [6]:
len(train_data_text), len(text_data)

(145000, 155070)

## Vectorizing the text data

We'll use the `TextVectorization` layer to vectorize the text data,
that is to say, to turn the
original strings into integer sequences where each integer represents the index of
a word in a vocabulary. We will use a custom string standardization scheme
(strip punctuation characters except `<` and `>`) and the default
splitting scheme (split on whitespace).

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~0123456789')
tokenizer.fit_on_texts(train_data_text)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
VOCAB_SIZE=0
out=0
for word, q in ast.literal_eval(tokenizer.get_config()['word_counts']).items():
  if q > 4: VOCAB_SIZE-=-1
  else: out-=-1
VOCAB_SIZE, out

(8056, 13313)

In [8]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")

vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
    pad_to_max_tokens=True
)
vectorization.adapt(train_data_text)


2025-10-05 11:16:42.512423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-05 11:16:42.513861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-10-05 11:16:42.513901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-10-05 11:16:42.513922: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-10-05 11:16:42.513942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [9]:

def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def process_input(img_path, captions):
    return decode_and_resize(img_path), vectorization(captions)


def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(len(images), reshuffle_each_iteration=True)
    dataset = dataset.map(process_input, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    return dataset

test_dataset = make_dataset(list(test_data.keys()), list(test_data.values()))

## Building the model

In [10]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [11]:

def get_cnn_model():
    base_model = EfficientNetV2L(
        input_shape=(*IMAGE_SIZE, 3), include_top=False, weights="imagenet",
    )
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(dense_dim, activation="relu")
        self.dense_2 = layers.Dense(embed_dim)
        self.dropout = layers.Dropout(rate)

    def call(self, inputs, training, mask=None):

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )
        
        out_1 = self.layernorm_1(inputs + attention_output_1)
        out_2 = self.dense_1(out_1)
        out_3 = self.dense_2(out_2) #512
        out_3 = self.dropout(out_3, training=training)
        
        return self.layernorm_2(out_1 + out_3)

class PositionalEmbedding_IMG(layers.Layer):
    def __init__(self, sequence_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.pos_encoding = positional_encoding(length=sequence_length, depth=embed_dim)
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim

    def call(self, x):
        length = tf.shape(x)[1]
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

class Encoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, num_layers, rate=0.4, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_layers = num_layers
        self.embedding = layers.Dense(embed_dim, activation='relu')
        self.pos_embedding = PositionalEmbedding_IMG(
            embed_dim=embed_dim, sequence_length=r_size * c_size
        )
        self.enc_layers = [TransformerEncoderBlock(embed_dim, dense_dim, num_heads) 
                       for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)
    def call(self, x, training, mask=None):
        x = self.embedding(x)  # (batch_size, input_seq_len(H*W), d_model) 512
        # length = tf.shape(x)[1]
        x = self.pos_embedding(x)
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
          x = self.enc_layers[i](x, training, mask)
        
        return x  # (batch_size, input_seq_len, d_model)

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_encoding = positional_encoding(length=sequence_length, depth=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.token_embeddings(x)
        x *= tf.math.sqrt(tf.cast(self.embed_dim, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.dropout_1 = layers.Dropout(0.8)
        self.dropout_2 = layers.Dropout(0.8)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2, attention_scores = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
            return_attention_scores=True
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.ffn_layer_2(ffn_out)
        ffn_out = self.dropout_2(ffn_out, training=training)
        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
        
        return ffn_out, attention_scores

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

class Decoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, n_layers, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.n_layers = n_layers
        self.layernorm_1 = layers.LayerNormalization()
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")
        self.embedding = PositionalEmbedding(
            embed_dim=embed_dim, sequence_length=SEQ_LENGTH, vocab_size=VOCAB_SIZE
        )
        self.decoder_layers = [TransformerDecoderBlock(embed_dim=embed_dim, ff_dim=FF_DIM, num_heads=heads)
                                for _ in range(n_layers)]
  def call(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)

        # def call(self, inputs, encoder_outputs, training, mask=None)
        for i in range(self.n_layers):
            inputs, weights = self.decoder_layers[i](inputs, encoder_outputs, training, mask)
        pred = self.out(inputs)
        return pred
  def pred(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)

        # def call(self, inputs, encoder_outputs, training, mask=None)
        layers_att_weights = []
        for i in range(self.n_layers):
            inputs, weights = self.decoder_layers[i](inputs, encoder_outputs, training, mask)
            layers_att_weights.append(weights)
        pred = self.out(inputs)
        return pred, layers_att_weights

class ImageCaptioningModel(keras.Model):
    def __init__(
        self, cnn_model, encoder, decoder, num_captions_per_image=5, image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        img_embed = self.cnn_model(batch_img)
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )

                batch_loss += loss
                batch_acc += acc

            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )

            grads = tape.gradient(loss, train_vars)

            self.optimizer.apply_gradients(zip(grads, train_vars))

        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        img_embed = self.cnn_model(batch_img)
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        return [self.loss_tracker, self.acc_tracker]


In [12]:
EMBED_DIM = 512
r_size, c_size=12, 12
FF_DIM = 2048

heads = 6
cnn_model = get_cnn_model()
encoder = Encoder(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=heads, num_layers=2)
decoder = Decoder(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=heads, n_layers=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=None,
)

In [13]:
# caption_model.load_weights('pesos/model.tf')
caption_model.load_weights('pesos/colab/modelV2/drive-download-20230331T190528Z-001/model_V2.43-13.6821.tf')

In [14]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,)
learning_rate = LRSchedule(post_warmup_learning_rate=5e-5, warmup_steps=8000)

In [15]:
caption_model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate,beta_1=0.9, beta_2=0.98,
                                                      epsilon=1e-9), loss=cross_entropy)

In [16]:
caption_model.evaluate(test_dataset)

2025-10-05 11:18:34.121922: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 117964800 exceeds 10% of free system memory.
2025-10-05 11:18:34.220794: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 117964800 exceeds 10% of free system memory.
2025-10-05 11:18:34.249090: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 117964800 exceeds 10% of free system memory.
2025-10-05 11:18:34.296277: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 117964800 exceeds 10% of free system memory.
2025-10-05 11:18:34.346600: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 117964800 exceeds 10% of free system memory.


40/40 [==============================] - 327s 8s/step - loss: 13.4732 - acc: 0.4031


[13.473180770874023, 0.40774327516555786]

## Check sample predictions

In [17]:
%matplotlib inline
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import matplotlib, einops
from tqdm import tqdm
from PIL import Image
from collections import defaultdict
import pandas as pd


matplotlib.use('agg')

In [18]:
vocab = vectorization.get_vocabulary()
index_lookup = dict(zip(range(len(vocab)), vocab))
max_decoded_sentence_length = SEQ_LENGTH - 1
test_images = list(test_data.keys())


def generate_caption(sample_img):
    attention_plot = defaultdict(list)
    sample_img = decode_and_resize(sample_img)
    img = sample_img.numpy().clip(0, 255).astype(np.uint8)

    img = tf.expand_dims(sample_img, 0)
    img = caption_model.cnn_model(img)

    encoded_img = caption_model.encoder(img, training=False)

    decoded_caption = "<start> "
    for i in range(max_decoded_sentence_length):
        tokenized_caption = vectorization([decoded_caption])[:, :-1]
        mask = tf.math.not_equal(tokenized_caption, 0)
        predictions, att_weights = caption_model.decoder.pred(
            tokenized_caption, encoded_img, training=False, mask=mask
        )
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = index_lookup[sampled_token_index]
        
        for j in range(heads):
          attention_plot[j].append(tf.squeeze(att_weights[-1],0)[j][i])
        if sampled_token == '[UNK]': continue
        decoded_caption += " " + sampled_token
        if sampled_token == "<end>":
            break
    decoded_caption = decoded_caption.replace("<start> ", "").strip()
    return decoded_caption, attention_plot

In [19]:
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.stats import median_abs_deviation as mad
from scipy.stats import iqr

In [20]:
def return_max(dict_tensor):
    tensors = list(attention_plot.values())
    return np.max(tensors)

In [22]:
stds, mads, iqrs = defaultdict(list), defaultdict(list), defaultdict(list)
d = defaultdict(list)

sentences = {}
save_figs = False
j = 1
layer=1
for image in tqdm(test_images):
  key = image.split('/')[4]
  if key not in sentences.keys():
      sentences[key] = {'references': [], 'candidate': None}
  sentences[key]['references'] = [caption.replace("<start> ", "").replace(" <end>", "").strip()
                                  for caption in test_data[image]]
  if sentences[key]['candidate'] is None:
      caption,attention_plot = generate_caption(image)
      sentences[key]['candidate'] = [caption]
      temp_image = np.array(Image.open(image))
      len_result = len(caption.split(' '))
      grid_size = max(int(np.ceil(len_result/2)), 2)
      for head in range(0, 6):
          if not os.path.isdir(f'flickr30k-attention-head{head}V2'):
            os.mkdir(f'flickr30k-attention-head{head}V2')
          if save_figs:
              fig = plt.figure(figsize=(20, 20), dpi=150)
              fig.tight_layout()
          for i in range(len(caption.split(' '))):
              temp_att = np.resize(attention_plot[head][i], (r_size, c_size))
              d[caption.split(' ')[i]+f'_{head}'].append(temp_att)
              std, mad_, iqr_ = np.std(temp_att.flatten()), mad(temp_att.flatten()), iqr(temp_att.flatten())
              stds[caption.split(' ')[i]+f'_{head}'].append(std)
              mads[caption.split(' ')[i]+f'_{head}'].append(mad_)
              iqrs[caption.split(' ')[i]+f'_{head}'].append(iqr_)
              if not save_figs: continue
              cap_folder = f'flickr30k-attention-head{head}V2/'+caption.split(' ')[i]
              if not os.path.isdir(f'flickr30k-attention-head{head}V2/'+caption.split(' ')[i]):
                  os.mkdir(f'flickr30k-attention-head{head}V2/'+caption.split(' ')[i])
              grid_size = min(1+int(np.ceil(len_result/2)), 5)
              ax = fig.add_subplot(grid_size, grid_size, i+1)
              plt.subplots_adjust(wspace=.7)
              ax.set_title(caption.split(' ')[i], fontsize=16, pad=2)
              ax.axis('off')
              norm = Normalize(vmin=0, vmax=100)
              img = ax.imshow(temp_image, aspect='auto')
              ax.imshow(temp_att, cmap='Greens', alpha=0.8, extent=img.get_extent(), aspect='auto',)
              cax = ax.inset_axes([1.05, .0, .08, 1])
              cbar=fig.colorbar(ScalarMappable(norm=norm, cmap='Greens'), cax=cax, orientation="vertical",
                           format="{x:.2f}")
              cbar.ax.tick_params(labelsize=15)
              cbar.set_ticks([0, 20, 40, 60, 80, 100])
              cbar.set_ticklabels(['0%', '20%', '40%', '60%', '80%', '100%'])
          if not save_figs: continue
          plt.savefig(f'flickr30k-attention-head{head}V2/{key.replace(".jpg", ".pdf")}',
                      bbox_inches='tight')
          plt.cla()
          plt.close('all')
          plt.clf()
#           break
#       break

100%|████████████████████████████████████████████████████████████████| 1000/1000 [19:30<00:00,  1.17s/it]


In [23]:
caption

'uma criança pequena joga na água <end>'

In [23]:
from PIL import Image

In [21]:
import seaborn as sns

In [24]:
for k, v in tqdm(d.items()):
    word, head = k.split('_')
    fig = plt.figure(figsize=(8, 8), dpi=100)
    fig.tight_layout()
    ax = fig.add_subplot()
    ax.imshow(np.mean(np.array(d[k]), axis=0), cmap='Greens')
    ax.axis('off')
    cax = ax.inset_axes([1.05, .0, .08, 1])
    norm = Normalize(vmin=0, vmax=100)
    cbar=fig.colorbar(ScalarMappable(norm=norm, cmap='Greens'), cax=cax, orientation="vertical",
               format="{x:.2f}")
    cbar.ax.tick_params(labelsize=30)
    cbar.set_ticks([0, 20, 40, 60, 80, 100])
    cbar.set_ticklabels(['0%', '20%', '40%', '60%', '80%', '100%'])
    plt.savefig(f"flickr30k-attention-head{head}V2/{word}/{word}_mean.png",
                      bbox_inches='tight')
    plt.cla()
    plt.close('all')
    plt.clf()
    
    f=None
    # break

  0%|                                                                           | 0/4188 [00:00<?, ?it/s]


In [19]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            # 👇️ alternatively use str()
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [28]:
with open('flickr_attentionheads_stdsV2.json', 'w') as jsonFile:
    json.dump(stds, jsonFile, indent=4, cls=NpEncoder)

In [29]:
with open('flickr_attentionheads_madsV2.json', 'w') as jsonFile:
    json.dump(mads, jsonFile, indent=4, cls=NpEncoder)

In [30]:
with open('flickr_attentionheads_iqrsV2.json', 'w') as jsonFile:
    json.dump(iqrs, jsonFile, indent=4, cls=NpEncoder)

In [20]:
with open('flickr_attentionheads_attentionmatrixV2.json', 'w') as jsonFile:
    json.dump(d, jsonFile, indent=4, cls=NpEncoder)

# Evaluation

In [27]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

class COCOEvalCap:
    def __init__(self,images,gts,res):
        self.evalImgs = []
        self.eval = {}
        self.imgToEval = {}
        self.params = {'image_id': images}
        self.gts = gts
        self.res = res

    def evaluate(self):
        imgIds = self.params['image_id']
        gts = self.gts
        res = self.res

        # =================================================
        # Set up scorers
        # =================================================
        print ('tokenization...')
        tokenizer = PTBTokenizer()
        gts  = tokenizer.tokenize(gts)
        res = tokenizer.tokenize(res)

        # =================================================
        # Set up scorers
        # =================================================
        print ('setting up scorers...')
        scorers = [
            (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
            (Meteor(),"METEOR"),
            (Rouge(), "ROUGE_L"),
            (Cider(), "CIDEr")
        ]

        # =================================================
        # Compute scores
        # =================================================
        eval = {}
        for scorer, method in scorers:
            print ('computing %s score...'%(scorer.method()))
            score, scores = scorer.compute_score(gts, res)
            if type(method) == list:
                for sc, scs, m in zip(score, scores, method):
                    self.setEval(sc, m)
                    self.setImgToEvalImgs(scs, imgIds, m)
                    print ("%s: %0.3f"%(m, sc))
            else:
                self.setEval(score, method)
                self.setImgToEvalImgs(scores, imgIds, method)
                print ("%s: %0.3f"%(method, score))
        self.setEvalImgs()

    def setEval(self, score, method):
        self.eval[method] = score

    def setImgToEvalImgs(self, scores, imgIds, method):
        for imgId, score in zip(imgIds, scores):
            if not imgId in self.imgToEval:
                self.imgToEval[imgId] = {}
                self.imgToEval[imgId]["image_id"] = imgId
            self.imgToEval[imgId][method] = score

    def setEvalImgs(self):
        self.evalImgs = [eval for imgId, eval in self.imgToEval.items()]

def calculate_metrics(rng,datasetGTS,datasetRES):
    imgIds = rng
    gts = {}
    res = {}

    imgToAnnsGTS = {ann['image_id']: [] for ann in datasetGTS['annotations']}
    for ann in datasetGTS['annotations']:
        imgToAnnsGTS[ann['image_id']] += [ann]

    imgToAnnsRES = {ann['image_id']: [] for ann in datasetRES['annotations']}
    for ann in datasetRES['annotations']:
        imgToAnnsRES[ann['image_id']] += [ann]

    for imgId in imgIds:
        gts[imgId] = imgToAnnsGTS[imgId]
        res[imgId] = imgToAnnsRES[imgId]

    evalObj = COCOEvalCap(imgIds,gts,res)
    evalObj.evaluate()
    return evalObj.eval

In [28]:
refs = []

sys = []

for j, image in enumerate(sentences.keys()):
    sys.append({'image_id':j, 'caption':sentences[image]['candidate'][0]})
    refslist = []
    for i, reference in enumerate(sentences[image]['references']):
        final_ref = ''
        r = reference.translate(str.maketrans('', '', string.punctuation)).lower()
        for word in r.split(' '):
            if word in vocab and word != '':
                final_ref += word + ' '
        refs.append({'image_id':j, 'caption':final_ref})

In [29]:
datasetGTS,datasetRES = {'annotations':refs}, {'annotations':sys}

In [30]:
calculate_metrics(range(1000),datasetGTS,datasetRES)

tokenization...


PTBTokenizer tokenized 67218 tokens at 828522.63 tokens per second.
PTBTokenizer tokenized 11607 tokens at 262566.44 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 10608, 'reflen': 10535, 'guess': [10608, 9608, 8608, 7609], 'correct': [6131, 2597, 1049, 417]}
ratio: 1.0069292833411478
Bleu_1: 0.578
Bleu_2: 0.395
Bleu_3: 0.267
Bleu_4: 0.180
computing METEOR score...
METEOR: 0.182
computing Rouge score...
ROUGE_L: 0.398
computing CIDEr score...
CIDEr: 0.360


{'Bleu_1': 0.5779600301658581,
 'Bleu_2': 0.39524681608136714,
 'Bleu_3': 0.2670156173739529,
 'Bleu_4': 0.17972339557087477,
 'METEOR': 0.18156448757285593,
 'ROUGE_L': 0.398117854946767,
 'CIDEr': 0.35982566779752767}